In [ ]:
# check name in male and female dictionaries
def CheckSex(name):
    if name in dict_male_rev:
        return 'm'
    elif name in dict_female_rev:
        return 'f'
    return '-'

In [ ]:
#check if there are records in whitch the winner_id e loser_id corresponds
def find_match_sameWL():
    df_sameWL = df_tennis[df_tennis['winner_id'] == df_tennis['loser_id']][['tourney_id','match_num','winner_id', 'loser_id', 'winner_name', 'loser_name','winner_ioc','loser_ioc']]
    print(df_sameWL.shape[0])
    return df_sameWL

In [ ]:
#check if there are ids binded with more then one name
def get_w_id_names():
    dup_w_id = df_tennis.groupby(by = 'winner_id')[['winner_name','winner_id']].nunique()
    return dup_w_id[dup_w_id['winner_name'] > 1].index.tolist()

In [ ]:
def get_l_id_names():
    dup_l_id = df_tennis.groupby(by = 'loser_id')[['loser_name','loser_id']].nunique()
    return dup_l_id[dup_l_id['loser_name'] > 1].index.tolist()

In [ ]:
def get_w_name_ids():
    dup_w_id = df_tennis.groupby(by = 'winner_name')[['winner_name','winner_id']].nunique()
    return dup_w_id[dup_w_id['winner_id'] > 1].index.tolist()

In [ ]:
def get_l_name_ids():
    dup_l_id = df_tennis.groupby(by = 'loser_name')[['loser_name','loser_id']].nunique()
    return dup_l_id[dup_l_id['loser_id'] > 1].index.tolist()

In [ ]:
def Retirement(s):
    if s.lower() in "retirement." or s.lower()=="ret.":
        return True
    else:
        return False
    
def Walkover(s):
    if s.lower()=="w/o" or s.lower()=="wo" or s.lower() in "walkover":
        return True
    else:
        return False
    
def Default(s):
    if s.lower() in "default." or s.lower()=="def.":
        return True
    else:
        return False
    
def Bye(s):
    if s.lower() in "bye.":
        return True
    else:
        return False

In [ ]:
import math

def validity_match(games, best_of):
    
    if len(games) == 0:
        return False
    
    if len(games)>best_of+1:
        return False
    
    if len(games)<math.trunc(best_of/2)+1 or len(games)>best_of:
        if not Retirement(games[len(games)-1]) and not Default(games[len(games)-1]):
            return False
        elif len(games)==1:
             return True
    
    first_player = 0
    second_player = 0
    
    if Retirement(games[len(games)-1]) or Default(games[len(games)-1]):
        #analysis of set with a ret or def
        game = games[len(games)-2].replace("[", "").replace("]", "")
        points = game.split("-")
        
        if "(" in game or ")" in game:
            points = game[0:(game.index("("))].split("-")
            if "6" not in points or ("7" not in points and points.count('6')!=2):
                return False
        else:
            x = int(points[0])
            y = int(points[1])

            if not ((max(x,y) <= 6 and min(x,y)<=6) or (max(x,y) == 7 and min(x,y)==5)):
                return False
        
        complete_sets = len(games)-2
    else:
        complete_sets = len(games)
    
    i = 0
    #analysis of complete sets
    while i < complete_sets:
        
        game = games[i].replace("[", "").replace("]", "")

        if "(" in game or ")" in game:
            #tie-break
            points = game[0:(game.index("("))].split("-")
            if "6" not in points or "7" not in points:
                return False
            else:
                if points[0] == "6":
                    second_player+=1
                else:
                    first_player+=1
                i+=1
        else:
            #no tie-break
            game = games[i].replace("[", "").replace("]", "")
            points = game.split("-")

            x = int(points[0])
            y = int(points[1])

            if max(x,y) == 7 and min(x,y)!=5:
                return False
            elif max(x,y)!= 6 and (max(x,y) != 7 or min(x,y)!=5):
                return False
            elif max(x,y)==6 and (min(x,y)>4 or min(x,y)<0):
                return False
            else:
                #valid case
                if x == min(x,y):
                    second_player+=1
                else:
                    first_player+=1
                i+=1

    #if the a player wins cannot be other matches
    if (first_player==math.trunc(best_of/2)+1 or second_player==math.trunc(best_of/2)+1) and \
        (second_player + first_player)!=len(games):
        return False

    return True
    

In [ ]:
def is_best_of_5(games):
    #not interesting case
    if len(games)!=3 and len(games)!=4:
        return False
    
    if Retirement(games[len(games)-1]) or Default(games[len(games)-1]):
        complete_sets = len(games)-2
    else:
        complete_sets = len(games)
    
    if complete_sets!=2 and complete_sets!=3:
        #not interesting case
        return False
    
    first_player = 0
    second_player = 0
    
    i = 0
    #analysis of complete sets
    while i < complete_sets:
        
        game = games[i].replace("[", "").replace("]", "")

        if "(" in game or ")" in game:
            #tie-break
            points = game[0:(game.index("("))].split("-")
        else:
            points = game.split("-")

        if int(points[0]) > int(points[1]):
            first_player+=1
        else:
            second_player+=1
        i+=1
            
    if first_player==0 or second_player==0:
        return True
            
    return False

In [ ]:
def is_best_of_3(games):
    
    #not interesting case
    if len(games)!=2 and len(games)!=3:
        return False
    
    if Retirement(games[len(games)-1]) or Default(games[len(games)-1]):
        return False
    
    first_player = 0
    second_player = 0
    
    i = 0
    #analysis of complete sets
    while i < len(games):
        
        game = games[i].replace("[", "").replace("]", "")

        if "(" in game or ")" in game:
            #tie-break
            points = game[0:(game.index("("))].split("-")
        else:
            points = game.split("-")

        if int(points[0]) > int(points[1]):
            first_player+=1
        else:
            second_player+=1
        i+=1
            
    if len(games)==2 and (first_player==0 or second_player==0):
        return True
    elif len(games)==3 and first_player!=0 and second_player!=0:
        return True
            
    return False

In [ ]:
## Print decimal age in a string years,month,days format
def ConvertAge(decimal_years):
    decimal_days = decimal_years * 365
    year = decimal_days//365
    tmp_days = decimal_days - (year*365)
    month = tmp_days//31
    days = tmp_days - (month * 31)
    return str(int(year)) + " years, " + str(int(month)) + " months, " + str(int(days)) + " days"

In [ ]:
def ConvertTime(time):
    hours = int((time // 60))
    minutes = int((time % 60))
    print(time % 1)
    seconds = int((time % 1) * 60)
    return ("%dH:%02dm:%02ds" % (hours, minutes, seconds))

In [ ]:
# we check that matches with 0 minutes are matches with BYE and WOLKOVER (in this case the match is not played)
def IsMatchWithZeroIncorrect(match):
    tmp = str(match)
    sets = tmp.split()    
    if len(sets)==1 and Walkover(sets[0]):
        return False
    if len(sets)==1 and Bye(sets[0]):
        return False
    if len(sets)==1 and Retirement(sets[0]):
        return False
    if len(sets)==1 and Default(sets[0]):
        return False
    return True

In [ ]:
def count_games(match):
    sets = match.split( )
    
    count = 0
    
    #empty score
    if len(sets)==0:
        return -1
    
    if len(sets)==1 and (Retirement(sets[0]) or Default(sets[0]) or Bye(sets[0]) or Walkover(sets[0])):
            return 0
    
    if Retirement(sets[len(sets)-1]) or Default(sets[len(sets)-1]):
        check_sets = len(sets)-1
    else:
        check_sets = len(sets)
    
    #analysis of complete sets
    for i in range(check_sets):
        
        game = sets[i].replace("[", "").replace("]", "")

        if "(" in game or ")" in game:
            #tie-break
            index_open = game.index("(")
            index_close = game.index(")")
            
            tie_break = game[index_open+1:index_close]
            if int(tie_break)<=5:
                count += int(tie_break) + 7 
            else:
                count += int(tie_break) * 2 + 2 
            
            points = game[0:index_open].split("-")
            
            #errors in the score
            if not points[0].isdigit() or not points[1].isdigit():
                return -1
            
            count += int(points[0]) + int(points[1])
            
        else:
            #no tie-break
            points = game.split("-")
            
            #errors in the score
            if not points[0].isdigit() or not points[1].isdigit():
                return -1
            
            count += int(points[0]) + int(points[1])
            
    return count

In [ ]:
def round_2(x):
    return float("{:.2f}".format(x))

In [ ]:
# Utility function to compute percentage
def percentage_ratio(total, partial):
    if (total <= 0): return -1
    return round(((partial*100)/total), 2)

In [ ]:
def count_service_games(match):
    sets = match.split( )
    
    player1 = 0
    player2 = 0
    
    first = 0
    
    #empty score
    if len(sets)==0:
        return [-1,-1]
    
    if len(sets)==1 and (Retirement(sets[0]) or Default(sets[0]) or Bye(sets[0]) or Walkover(sets[0])):
            return [-1,-1]
    
    if Retirement(sets[len(sets)-1]) or Default(sets[len(sets)-1]):
        check_sets = len(sets)-1
    else:
        check_sets = len(sets)
    
    #analysis of complete sets
    for i in range(check_sets):
        
        count = 0
        
        game = sets[i].replace("[", "").replace("]", "")

        if "(" in game or ")" in game:
            #tie-break
            index_open = game.index("(")
            
            points = game[0:index_open].split("-")
            
            #errors in the score
            if not points[0].isdigit() or not points[1].isdigit():
                return [-1,-1]
            
            count = int(points[0]) + int(points[1]) -1
            
            x = count//2
            y = count - x #y>=x

            if first == 0:
                player1 += y  
                player2 += x
            else:
                player1 += x  
                player2 += y
                
            first = (first+1)%2
            
        else:
            #no tie-break
            points = game.split("-")
            
            #errors in the score
            if not points[0].isdigit() or not points[1].isdigit():
                return [-1,-1]
            
            count = int(points[0]) + int(points[1])
            
            x = count//2
            y = count - x #y>=x

            if first == 0:
                player1 += y  
                player2 += x
            else:
                player1 += x  
                player2 += y
                
            if count%2 != 0:
                first = (first+1)%2
            
        
    return [player1, player2]